In [1]:
import numpy as np
# import gdal
import rasterio
import scipy
import os
import glob
import pandas as pd
from numpy import expand_dims
from numpy import moveaxis
from multiprocessing import Pool
import time

In [2]:
# Read all temporal images from dir using rasterio and append to list 
images = []
path = "final_img_master/*.tif"
for filename in sorted(glob.iglob(path)):
    print(filename)
    with rasterio.open(filename,'r') as ds:
        arr = ds.read()
    data = moveaxis(arr, 0, 2)
    images.append(data)

final_img_master/S1A_IW_GRDH_1SDV_20180104.tif
final_img_master/S1A_IW_GRDH_1SDV_20180105.tif
final_img_master/S1A_IW_GRDH_1SDV_20180107.tif
final_img_master/S1A_IW_GRDH_1SDV_20180109.tif
final_img_master/S1A_IW_GRDH_1SDV_20180110.tif
final_img_master/S1A_IW_GRDH_1SDV_20180111.tif
final_img_master/S1A_IW_GRDH_1SDV_20180112.tif
final_img_master/S1A_IW_GRDH_1SDV_20180116.tif
final_img_master/S1A_IW_GRDH_1SDV_20180117.tif
final_img_master/S1A_IW_GRDH_1SDV_20180119.tif
final_img_master/S1A_IW_GRDH_1SDV_20180121.tif
final_img_master/S1A_IW_GRDH_1SDV_20180122.tif
final_img_master/S1A_IW_GRDH_1SDV_20180123.tif
final_img_master/S1A_IW_GRDH_1SDV_20180124.tif
final_img_master/S1A_IW_GRDH_1SDV_20180128.tif
final_img_master/S1A_IW_GRDH_1SDV_20180129.tif
final_img_master/S1A_IW_GRDH_1SDV_20180131.tif
final_img_master/S1A_IW_GRDH_1SDV_20180202.tif
final_img_master/S1A_IW_GRDH_1SDV_20180203.tif
final_img_master/S1A_IW_GRDH_1SDV_20180204.tif
final_img_master/S1A_IW_GRDH_1SDV_20180205.tif
final_img_mas

In [4]:
# Convert to absolute values and reshape the data accodingly (rowxcolxbandsxseq_len)
images = np.array(images)
images = np.abs(images)
images = np.where(images == -32768,32767, images)
images = moveaxis(images,0,3)

In [5]:
# Interpolation similar to training data
def interpolate(pixel):
    seq_len = 222
    arr_T = pixel
    df1 = pd.DataFrame(arr_T[0])
    df2 = pd.DataFrame(arr_T[1])
    df3 = pd.DataFrame(arr_T[2])
    df1 = df1.replace(0,np.nan)
    df2 = df2.replace(0,np.nan)
    df3 = df3.replace(0,np.nan)
    df1.interpolate(method = 'linear',inplace=True)
    df1.replace(to_replace=np.nan, method='bfill',inplace=True)
    df2.interpolate(method = 'linear',inplace=True)
    df2.replace(to_replace=np.nan, method='bfill',inplace=True)
    df3.interpolate(method = 'linear',inplace=True)
    df3.replace(to_replace=np.nan, method='bfill',inplace=True)
    df1 = df1.values.reshape((seq_len,))
    df2 = df2.values.reshape((seq_len,))
    df3 = df3.values.reshape((seq_len,))
    pixel[0] = df1
    pixel[1] = df2
    pixel[2] = df3
    return pixel

In [6]:
t1 = time.time()
p = Pool()
result_arr = p.map(interpolate,[images[i][j] for i in range(len(images)) for j in range(len(images[0]))])
p.close()
p.join()
print("Took:",time.time()-t1)

Took: 8840.537841796875


In [9]:
# Save the interpolated test data
interpolated_data = np.array(result_arr)
interpolated_data = interpolated_data.reshape(images.shape[0],images.shape[1],images.shape[2],images.shape[3])
with open('interpolatedStudyRegion.npy', 'wb') as f:
    np.save(f,interpolated_data)